In [49]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import calendar
import warnings
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import missingno as mano
import re
from datetime import timedelta
warnings.filterwarnings('ignore')

In [2]:
#Importing the data

data =  pd.read_csv("C:/Users/Omar Hamzic/Documents/Project_1_Customer_Propensity_Model/data/data/final_customer_data_with_RFM.csv", parse_dates = ['DateTime'])
data.head()

,User_id,Session_id,DateTime,Category,SubCategory,Action,Quantity,Rate,Total Price,Recency,Frequency,Monetary,R,F,M,Group,Score,Loyalty_Level
0,5.224384e+10,d76fde-8bb3-4e00-8c23,2019-01-10 10:20:00,Electronic Appliances,Speakers,first_app_open,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5.224384e+10,33dfbd-b87a-4708-9857,2019-01-10 10:22:00,Electronic Appliances,Speakers,search,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5.731416e+10,6511c2-e2e3-422b-b695,2019-01-10 14:00:00,Men's Fashion,Jeans,search,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5.731416e+10,90fc70-0e80-4590-96f3,2019-01-10 14:07:00,Men's Fashion,Jeans,product_view,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5.731416e+10,bd7419-2748-4c56-95b4,2019-01-10 14:12:00,Men's Fashion,Jeans,read_reviews,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2090 entries, 0 to 2089
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   User_id        2090 non-null   float64       
 1   Session_id     2090 non-null   object        
 2   DateTime       2090 non-null   datetime64[ns]
 3   Category       2090 non-null   object        
 4   SubCategory    2090 non-null   object        
 5   Action         2090 non-null   object        
 6   Quantity       201 non-null    float64       
 7   Rate           201 non-null    float64       
 8   Total Price    201 non-null    float64       
 9   Recency        1303 non-null   float64       
 10  Frequency      1303 non-null   float64       
 11  Monetary       1303 non-null   float64       
 12  R              1303 non-null   float64       
 13  F              1303 non-null   float64       
 14  M              1303 non-null   float64       
 15  Group          1303 n

In [6]:
print(len(data.Action.value_counts()))
data['Action'] = data['Action'].apply(lambda x: 'read_reviews' if x in ('read_review') else ('add_to_wishlist' if x in ('add_to_wishist') else x))
print(len(data.Action.value_counts()))

11
11


In [8]:
data['Date'] = data['DateTime'].dt.date
data['DayOfWeek']= data['DateTime'].dt.dayofweek
data['DayOfMonth'] = data['DateTime'].dt.day
data.head()

,User_id,Session_id,DateTime,Category,SubCategory,Action,Quantity,Rate,Total Price,Recency,...,Monetary,R,F,M,Group,Score,Loyalty_Level,Date,DayOfWeek,DayOfMonth
0,5.224384e+10,d76fde-8bb3-4e00-8c23,2019-01-10 10:20:00,Electronic Appliances,Speakers,first_app_open,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-10,3,10
1,5.224384e+10,33dfbd-b87a-4708-9857,2019-01-10 10:22:00,Electronic Appliances,Speakers,search,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-10,3,10
2,5.731416e+10,6511c2-e2e3-422b-b695,2019-01-10 14:00:00,Men's Fashion,Jeans,search,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-10,3,10
3,5.731416e+10,90fc70-0e80-4590-96f3,2019-01-10 14:07:00,Men's Fashion,Jeans,product_view,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-10,3,10
4,5.731416e+10,bd7419-2748-4c56-95b4,2019-01-10 14:12:00,Men's Fashion,Jeans,read_reviews,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-10,3,10


## **Feature Engineering**

Feature engineering is the process of creating new, informative, and relevant features (variables) from existing data in a way that enhances the performance of machine learning models. It plays a crucial role in data preprocessing and can significantly impact the accuracy and effectiveness of predictive models. 


**Here we will be focus on feature engineering. This focuse will be on User Actvity features, focusing on:**
   > -Number of Days Active


   > -Average Time between purchase


   > -Cart to Purchase Ratio


   > -Wishlisht to Purchase Ratio


   > -Path to cart

   

**Number of Days Active**

In [9]:
## of days active
df=data.copy()
days_active = df.groupby('User_id').agg({'Date': 'nunique'}).reset_index()
days_active.rename(columns = {'Date': 'no_of_days_active'}, inplace=True)
days_active.head(5)

,User_id,no_of_days_active
0,5.144382e+10,1
1,5.144816e+10,3
2,5.145551e+10,3
3,5.146260e+10,5
4,5.146362e+10,3


**Average Time in between purchases**

In [17]:
#Average Time in between purchases
temp = df.copy()

def purch_time_diff(x):
    n = len(x)
    if n < 2:
        return None  # Not enough purchases to calculate the average time difference
    total_days = 0
    for i in range(1, n):
        total_days += (x[i] - x[i-1]).days
    return total_days / (n-1)

In [19]:
user_purch_date= temp[temp['Action'] == 'purchase'].sort_values(by='Date').groupby('User_id').agg({'Date' : lambda x: list(x)}).reset_index()

user_purch_date['purchase1'] = user_purch_date['Date'].apply(lambda x: x[0])
user_purch_date['purchase2'] = user_purch_date['Date'].apply(lambda x: x[1] if len(x)>1 else 0)
user_purch_date['purchase3'] = user_purch_date['Date'].apply(lambda x: x[2] if len(x)>2 else 0)
user_purch_date['avg_time_between_purchase'] = user_purch_date['Date'].apply(purch_time_diff)

print(user_purch_date.shape)
user_purch_date.head()

(167, 6)


,User_id,Date,purchase1,purchase2,purchase3,avg_time_between_purchase
0,5.144816e+10,"[2019-06-11, 2019-12-12]",2019-06-11,2019-12-12,0,184.0
1,5.145551e+10,[2019-10-19],2019-10-19,0,0,NaN
2,5.146260e+10,"[2019-10-20, 2019-12-15]",2019-10-20,2019-12-15,0,56.0
3,5.146362e+10,"[2019-10-14, 2019-11-12]",2019-10-14,2019-11-12,0,29.0
4,5.146553e+10,"[2019-12-05, 2019-12-12]",2019-12-05,2019-12-12,0,7.0


**Creating a Cart to Purchase Ratio**

In [43]:
#Creating a Cart to Purchase Ratio
del temp
temp = df.copy()
def purchase_ratios(action_col='add_to_cart', id_col='User_id', col_name='no_of_carts'):
    test = temp[temp['Action'] == action_col].groupby(id_col).agg({
        'Session_id': 'count',
        'Action': lambda x: (x == 'purchase').sum()
    }).reset_index()
    test.rename(columns={'Session_id': col_name, 'Action': 'no_of_purch'}, inplace=True)

    test_ratio = test.replace(np.nan, 0)
    test_ratio[col_name + '_to_purchase_ratio'] = (test_ratio['no_of_purch'] / test_ratio[col_name])

    print(test_ratio.shape)
    return test_ratio




In [44]:
cart_purch_ratio = purchase_ratios(action_col ='add_to_cart', id_col = 'User_id', col_name = 'no_of_carts')
display(cart_purch_ratio)
cart_purch_ratio.describe()

(240, 4)


,User_id,no_of_carts,no_of_purch,no_of_carts_to_purchase_ratio
0,5.144816e+10,2,0,0.0
1,5.145551e+10,1,0,0.0
2,5.146260e+10,2,0,0.0
3,5.146362e+10,2,0,0.0
4,5.146553e+10,3,0,0.0
...,...,...,...,...
235,6.003433e+10,1,0,0.0
236,6.008939e+10,1,0,0.0
237,6.023828e+10,3,0,0.0
238,6.030026e+10,1,0,0.0


,User_id,no_of_carts,no_of_purch,no_of_carts_to_purchase_ratio
count,2.400000e+02,240.000000,240.0,240.0
mean,5.396592e+10,1.166667,0.0,0.0
std,2.767010e+09,0.405678,0.0,0.0
min,5.144816e+10,1.000000,0.0,0.0
25%,5.185721e+10,1.000000,0.0,0.0
50%,5.231850e+10,1.000000,0.0,0.0
75%,5.586744e+10,1.000000,0.0,0.0
max,6.030264e+10,3.000000,0.0,0.0


In [45]:
temp = df.copy()

**wishlist to purchase ratio**

In [46]:
#Code looks at clicking wishlist to purchase ratio

click_wishlist_purch_ratio_user = purchase_ratios(action_col = 'click_wishlist_page', id_col = 'User_id', col_name = 'no_of_click_wishlist')
print(click_wishlist_purch_ratio_user.shape)
display(click_wishlist_purch_ratio_user.head())
click_wishlist_purch_ratio_user.describe()


(191, 4)
(191, 4)


,User_id,no_of_click_wishlist,no_of_purch,no_of_click_wishlist_to_purchase_ratio
0,5.144816e+10,1,0,0.0
1,5.146362e+10,1,0,0.0
2,5.146553e+10,1,0,0.0
3,5.146852e+10,1,0,0.0
4,5.147515e+10,1,0,0.0


,User_id,no_of_click_wishlist,no_of_purch,no_of_click_wishlist_to_purchase_ratio
count,1.910000e+02,191.000000,191.0,191.0
mean,5.403865e+10,1.057592,0.0,0.0
std,2.800753e+09,0.274978,0.0,0.0
min,5.144816e+10,1.000000,0.0,0.0
25%,5.182071e+10,1.000000,0.0,0.0
50%,5.230498e+10,1.000000,0.0,0.0
75%,5.618519e+10,1.000000,0.0,0.0
max,6.030264e+10,3.000000,0.0,0.0


In [38]:
#Code looks at adding to wishlist to purchase ratio

add_wishlist_purch_ratio_user = purchase_ratios(action_col = 'add_to_wishlist', id_col = 'User_id', col_name = 'no_of_add_wishlist')
print(add_wishlist_purch_ratio_user.shape)
display(add_wishlist_purch_ratio_user.head())
add_wishlist_purch_ratio_user.describe()

(26, 4)
(26, 4)


,User_id,no_of_add_wishlist,no_of_purch,no_of_add_wishlist_to_purchase_ratio
0,5.146362e+10,1,0,0.0
1,5.147745e+10,2,0,0.0
2,5.151800e+10,2,0,0.0
3,5.152646e+10,1,0,0.0
4,5.156516e+10,4,0,0.0


,User_id,no_of_add_wishlist,no_of_purch,no_of_add_wishlist_to_purchase_ratio
count,2.600000e+01,26.000000,26.0,26.0
mean,5.295457e+10,1.461538,0.0,0.0
std,2.529052e+09,0.760567,0.0,0.0
min,5.146362e+10,1.000000,0.0,0.0
25%,5.161642e+10,1.000000,0.0,0.0
50%,5.175573e+10,1.000000,0.0,0.0
75%,5.230051e+10,2.000000,0.0,0.0
max,5.953342e+10,4.000000,0.0,0.0


In [47]:
del temp
temp = df.copy()

**Added to cart**

In [48]:
#Filter for users who have added to cart

temp = (temp.sort_values(by ='DateTime')
        [(temp.DateTime <= temp.User_id.map(temp[temp['Action'] == 'add_to_cart'].groupby('User_id').DateTime.max()))])
print(temp.shape)
print(temp.User_id.nunique())
temp.head(3)

(1192, 21)
240


,User_id,Session_id,DateTime,Category,SubCategory,Action,Quantity,Rate,Total Price,Recency,...,Monetary,R,F,M,Group,Score,Loyalty_Level,Date,DayOfWeek,DayOfMonth
942,5.172531e+10,hfg03b-d558-4953-b218,2019-01-01 11:00:00,Men's Fashion,Shirts,search,NaN,NaN,NaN,24.0,...,4750.0,4.0,4.0,3.0,443.0,11.0,Platinum,2019-01-01,1,1
943,5.172531e+10,cbvg11b4-fadf-4d3c-8834,2019-01-01 11:05:00,Men's Fashion,Shirts,product_view,NaN,NaN,NaN,24.0,...,4750.0,4.0,4.0,3.0,443.0,11.0,Platinum,2019-01-01,1,1
944,5.172531e+10,gcv0ec1-a246-4d24-b94a,2019-01-01 11:10:00,Men's Fashion,Shirts,read_reviews,NaN,NaN,NaN,24.0,...,4750.0,4.0,4.0,3.0,443.0,11.0,Platinum,2019-01-01,1,1


In [73]:
temp_path = temp.sort_values(by='DateTime').groupby('User_id').agg({'Action': lambda x: list(x)}).reset_index()

last_3_actions = r'((?:\S+\s+){0,3}\badd_to_cart)'
temp_path = temp.sort_values(by='DateTime').groupby('User_id').agg({'Action': lambda x: list(x)}).reset_index()
temp_path['Action_cleaned'] = temp_path['Action'].apply(lambda x: ' '.join(x))
temp_path['Last_3_Actions'] = temp_path['Action_cleaned'].apply(lambda x: re.findall(last_3_actions,x))

#Cleaning and selecting the latest path for each user
temp_path['Final_3_Actions'] = temp_path['Last_3_Actions'].apply(lambda x: x[0] if len(x) ==1 else x)
temp_path['Final_3_Actions'] = temp_path['Final_3_Actions'].apply(lambda x: x[1] if len(x) == 2 else x)
temp_path['Final_3_Actions'] = temp_path['Final_3_Actions'].apply(lambda x: x[2] if len(x) == 3 else x)

print(temp_path.shape)
temp_path.head()

(240, 5)


,User_id,Action,Action_cleaned,Last_3_Actions,Final_3_Actions
0,5.144816e+10,"[first_app_open, search, product_view, read_re...",first_app_open search product_view read_review...,"[search product_view read_reviews add_to_cart,...",search read_reviews add_to_cart
1,5.145551e+10,"[first_app_open, search, product_view, read_re...",first_app_open search product_view read_review...,[search product_view read_reviews add_to_cart],search product_view read_reviews add_to_cart
2,5.146260e+10,"[search, product_view, read_reviews, add_to_ca...",search product_view read_reviews add_to_cart s...,"[search product_view read_reviews add_to_cart,...",search product_view add_to_cart
3,5.146362e+10,"[first_app_open, search, read_reviews, search,...",first_app_open search read_reviews search prod...,[purchase search click_wishlist_page add_to_ca...,search product_view add_to_cart
4,5.146553e+10,"[search, add_to_cart, click_wishlist_page, add...",search add_to_cart click_wishlist_page add_to_...,[search add_to_cart click_wishlist_page add_to...,search add_to_cart


In [75]:
temp_path = temp.sort_values(by='DateTime').groupby('User_id').agg({'Action': lambda x: list(x)}).reset_index()

def extract_last_3_actions(x):
    n = len(x)
    if n == 0:
        return []
    elif n <= 3:
        return x
    else:
        return x[-3:]

temp_path['Last_3_Actions'] = temp_path['Action'].apply(extract_last_3_actions)

temp_path['Action'] = temp_path['Action'].apply(' '.join)
print(temp_path.shape)
temp_path.head()


(240, 3)


,User_id,Action,Last_3_Actions
0,5.144816e+10,first_app_open search product_view read_review...,"[search, read_reviews, add_to_cart]"
1,5.145551e+10,first_app_open search product_view read_review...,"[product_view, read_reviews, add_to_cart]"
2,5.146260e+10,search product_view read_reviews add_to_cart s...,"[search, product_view, add_to_cart]"
3,5.146362e+10,first_app_open search read_reviews search prod...,"[search, product_view, add_to_cart]"
4,5.146553e+10,search add_to_cart click_wishlist_page add_to_...,"[add_to_cart, search, add_to_cart]"


In [76]:
temp_path.Last_3_Actions.value_counts()

[read_reviews, click_wishlist_page, add_to_cart]       64
[product_view, read_reviews, add_to_cart]              30
[add_to_cart]                                          24
[click_wishlist_page, add_to_cart]                     24
[product_view, click_wishlist_page, add_to_cart]       21
[search, read_reviews, add_to_cart]                    12
[search, product_view, add_to_cart]                    12
[read_reviews, add_to_cart]                             9
[search, click_wishlist_page, add_to_cart]              6
[checkout, purchase, add_to_cart]                       5
[read_reviews, product_view, add_to_cart]               3
[product_view, add_to_cart]                             3
[product_review, read_reviews, add_to_cart]             2
[add_to_wishlist, click_wishlist_page, add_to_cart]     2
[search, product_review, add_to_cart]                   2
[search, add_to_cart]                                   2
[read_reviews, add_to_cart, checkout]                   2
[purchase, rea

In [80]:
top_10_paths = temp_path.Last_3_Actions.value_counts()[:10].index.to_list()
temp_path['top_paths'] = temp_path['Last_3_Actions'].apply(lambda x : x if x in top_10_paths else 'others')
temp_path.head()
temp_path.top_paths.value_counts()

[read_reviews, click_wishlist_page, add_to_cart]    64
others                                              33
[product_view, read_reviews, add_to_cart]           30
[click_wishlist_page, add_to_cart]                  24
[add_to_cart]                                       24
[product_view, click_wishlist_page, add_to_cart]    21
[search, read_reviews, add_to_cart]                 12
[search, product_view, add_to_cart]                 12
[read_reviews, add_to_cart]                          9
[search, click_wishlist_page, add_to_cart]           6
[checkout, purchase, add_to_cart]                    5
Name: top_paths, dtype: int64

Looking at Category/Subcateogry level features

In [81]:
del temp

In [87]:
temp = df.copy()
temp.head()

,User_id,Session_id,DateTime,Category,SubCategory,Action,Quantity,Rate,Total Price,Recency,...,Monetary,R,F,M,Group,Score,Loyalty_Level,Date,DayOfWeek,DayOfMonth
0,5.224384e+10,d76fde-8bb3-4e00-8c23,2019-01-10 10:20:00,Electronic Appliances,Speakers,first_app_open,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-10,3,10
1,5.224384e+10,33dfbd-b87a-4708-9857,2019-01-10 10:22:00,Electronic Appliances,Speakers,search,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-10,3,10
2,5.731416e+10,6511c2-e2e3-422b-b695,2019-01-10 14:00:00,Men's Fashion,Jeans,search,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-10,3,10
3,5.731416e+10,90fc70-0e80-4590-96f3,2019-01-10 14:07:00,Men's Fashion,Jeans,product_view,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-10,3,10
4,5.731416e+10,bd7419-2748-4c56-95b4,2019-01-10 14:12:00,Men's Fashion,Jeans,read_reviews,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-10,3,10


In [89]:
cart_purch_ratio_category = purchase_ratios(action_col = 'add_to_cart', id_col = 'Category', col_name = 'category_no_of_carts')
display(cart_purch_ratio_category.head())
cart_purch_ratio_category.describe()

(89, 4)


,Category,category_no_of_carts,no_of_purch,category_no_of_carts_to_purchase_ratio
0,Accessories,15,0,0.0
1,Air Conditioner,1,0,0.0
2,Appliances,1,0,0.0
3,Artwork,1,0,0.0
4,Audio,1,0,0.0


,category_no_of_carts,no_of_purch,category_no_of_carts_to_purchase_ratio
count,89.000000,89.0,89.0
mean,3.146067,0.0,0.0
std,3.540514,0.0,0.0
min,1.000000,0.0,0.0
25%,1.000000,0.0,0.0
50%,2.000000,0.0,0.0
75%,3.000000,0.0,0.0
max,16.000000,0.0,0.0


In [91]:
cart_purch_ratio_subcategory = purchase_ratios(action_col = 'add_to_cart', id_col = 'SubCategory', col_name = 'subcategory_no_of_carts')
display(cart_purch_ratio_subcategory.head())
cart_purch_ratio_subcategory.describe()

(239, 4)


,SubCategory,subcategory_no_of_carts,no_of_purch,subcategory_no_of_carts_to_purchase_ratio
0,Air Tight Container,1,0,0.0
1,Air coolers,1,0,0.0
2,Amazon Alexa,1,0,0.0
3,Amplifier,1,0,0.0
4,Audio Speakers,1,0,0.0


,subcategory_no_of_carts,no_of_purch,subcategory_no_of_carts_to_purchase_ratio
count,239.000000,239.0,239.0
mean,1.171548,0.0,0.0
std,0.654546,0.0,0.0
min,1.000000,0.0,0.0
25%,1.000000,0.0,0.0
50%,1.000000,0.0,0.0
75%,1.000000,0.0,0.0
max,8.000000,0.0,0.0


In [92]:
del temp
temp = df.copy()
temp.head()

,User_id,Session_id,DateTime,Category,SubCategory,Action,Quantity,Rate,Total Price,Recency,...,Monetary,R,F,M,Group,Score,Loyalty_Level,Date,DayOfWeek,DayOfMonth
0,5.224384e+10,d76fde-8bb3-4e00-8c23,2019-01-10 10:20:00,Electronic Appliances,Speakers,first_app_open,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-10,3,10
1,5.224384e+10,33dfbd-b87a-4708-9857,2019-01-10 10:22:00,Electronic Appliances,Speakers,search,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-10,3,10
2,5.731416e+10,6511c2-e2e3-422b-b695,2019-01-10 14:00:00,Men's Fashion,Jeans,search,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-10,3,10
3,5.731416e+10,90fc70-0e80-4590-96f3,2019-01-10 14:07:00,Men's Fashion,Jeans,product_view,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-10,3,10
4,5.731416e+10,bd7419-2748-4c56-95b4,2019-01-10 14:12:00,Men's Fashion,Jeans,read_reviews,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-10,3,10


In [93]:
click_wishlist_purch_ratio_category = purchase_ratios(action_col = 'click_wishlist_page', id_col = 'Category', col_name = 'category_no_of_click_wishilist')
print(click_wishlist_purch_ratio_category.shape)
display(click_wishlist_purch_ratio_category.head())
click_wishlist_purch_ratio_category.describe()

(72, 4)
(72, 4)


,Category,category_no_of_click_wishilist,no_of_purch,category_no_of_click_wishilist_to_purchase_ratio
0,Accessories,14,0,0.0
1,Appliances,3,0,0.0
2,Artwork,1,0,0.0
3,Audio,1,0,0.0
4,Baby Care,4,0,0.0


,category_no_of_click_wishilist,no_of_purch,category_no_of_click_wishilist_to_purchase_ratio
count,72.000000,72.0,72.0
mean,2.805556,0.0,0.0
std,2.548896,0.0,0.0
min,1.000000,0.0,0.0
25%,1.000000,0.0,0.0
50%,2.000000,0.0,0.0
75%,4.000000,0.0,0.0
max,14.000000,0.0,0.0


In [94]:
click_wishlist_purch_ratio_subcategory = purchase_ratios(action_col = 'click_wishlist_page', id_col = 'SubCategory', col_name = 'subcategory_no_of_click_wishilist')
print(click_wishlist_purch_ratio_subcategory.shape)
display(click_wishlist_purch_ratio_subcategory.head())
click_wishlist_purch_ratio_subcategory.describe()

(175, 4)
(175, 4)


,SubCategory,subcategory_no_of_click_wishilist,no_of_purch,subcategory_no_of_click_wishilist_to_purchase_ratio
0,Air Tight Container,1,0,0.0
1,Air coolers,1,0,0.0
2,Amazon Alexa,1,0,0.0
3,Audio Speakers,1,0,0.0
4,Baby Shampoo,1,0,0.0


,subcategory_no_of_click_wishilist,no_of_purch,subcategory_no_of_click_wishilist_to_purchase_ratio
count,175.000000,175.0,175.0
mean,1.154286,0.0,0.0
std,0.581488,0.0,0.0
min,1.000000,0.0,0.0
25%,1.000000,0.0,0.0
50%,1.000000,0.0,0.0
75%,1.000000,0.0,0.0
max,6.000000,0.0,0.0


In [95]:
add_wishlist_purch_ratio_category = purchase_ratios(action_col = 'add_to_wishlist', id_col = 'Category', col_name = 'category_no_of_add_wishilist')
print(add_wishlist_purch_ratio_category.shape)
display(add_wishlist_purch_ratio_category.head())
add_wishlist_purch_ratio_category.describe()

(18, 4)
(18, 4)


,Category,category_no_of_add_wishilist,no_of_purch,category_no_of_add_wishilist_to_purchase_ratio
0,Accessories,3,0,0.0
1,Baby Care,1,0,0.0
2,Bed room,1,0,0.0
3,Digital Appliances,1,0,0.0
4,Digital Devices,5,0,0.0


,category_no_of_add_wishilist,no_of_purch,category_no_of_add_wishilist_to_purchase_ratio
count,18.000000,18.0,18.0
mean,2.111111,0.0,0.0
std,1.529663,0.0,0.0
min,1.000000,0.0,0.0
25%,1.000000,0.0,0.0
50%,1.500000,0.0,0.0
75%,2.750000,0.0,0.0
max,6.000000,0.0,0.0


In [96]:
add_wishlist_purch_ratio_subcategory = purchase_ratios(action_col = 'add_to_wishlist', id_col = 'SubCategory', col_name = 'subcategory_no_of_add_wishilist')
print(add_wishlist_purch_ratio_subcategory.shape)
display(add_wishlist_purch_ratio_subcategory.head())
add_wishlist_purch_ratio_subcategory.describe()

(26, 4)
(26, 4)


,SubCategory,subcategory_no_of_add_wishilist,no_of_purch,subcategory_no_of_add_wishilist_to_purchase_ratio
0,Books,2,0,0.0
1,Bp monitors,1,0,0.0
2,Bumpers,1,0,0.0
3,Cases,3,0,0.0
4,Charging wire,1,0,0.0


,subcategory_no_of_add_wishilist,no_of_purch,subcategory_no_of_add_wishilist_to_purchase_ratio
count,26.000000,26.0,26.0
mean,1.461538,0.0,0.0
std,0.706018,0.0,0.0
min,1.000000,0.0,0.0
25%,1.000000,0.0,0.0
50%,1.000000,0.0,0.0
75%,2.000000,0.0,0.0
max,3.000000,0.0,0.0


In [119]:
add_product_view_purch_ratio_category = purchase_ratios(action_col = 'product_view', id_col = 'Category', col_name = 'category_no_of_product_view')
print(add_product_view_purch_ratio_category.shape)
display(add_product_view_purch_ratio_category.head())
add_product_view_purch_ratio_category.describe()

add_product_view_purch_ratio_subcategory = purchase_ratios(action_col = 'product_view', id_col = 'SubCategory', col_name = 'subcategory_no_of_product_view')
print(add_product_view_purch_ratio_subcategory.shape)
display(add_product_view_purch_ratio_subcategory.head())
add_product_view_purch_ratio_subcategory.describe()


(90, 4)
(90, 4)


,Category,category_no_of_product_view,no_of_purch,category_no_of_product_view_to_purchase_ratio
0,Accessories,9,0,0.0
1,Air Conditioner,7,0,0.0
2,Artwork,1,0,0.0
3,Audio,1,0,0.0
4,Baby Care,3,0,0.0


(216, 4)
(216, 4)


,SubCategory,subcategory_no_of_product_view,no_of_purch,subcategory_no_of_product_view_to_purchase_ratio
0,Amazon Alexa,1,0,0.0
1,Arm sleeve,1,0,0.0
2,Aux Cable,3,0,0.0
3,Baby Shampoo,1,0,0.0
4,Baby Shower gel,1,0,0.0


,subcategory_no_of_product_view,no_of_purch,subcategory_no_of_product_view_to_purchase_ratio
count,216.000000,216.0,216.0
mean,1.351852,0.0,0.0
std,0.860663,0.0,0.0
min,1.000000,0.0,0.0
25%,1.000000,0.0,0.0
50%,1.000000,0.0,0.0
75%,1.000000,0.0,0.0
max,7.000000,0.0,0.0


Modeling Dataset Creation:

In [148]:
#del df_base

In [149]:
print(df.shape)
df.head()

(2090, 21)


,User_id,Session_id,DateTime,Category,SubCategory,Action,Quantity,Rate,Total Price,Recency,...,Monetary,R,F,M,Group,Score,Loyalty_Level,Date,DayOfWeek,DayOfMonth
0,5.224384e+10,d76fde-8bb3-4e00-8c23,2019-01-10 10:20:00,Electronic Appliances,Speakers,first_app_open,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-10,3,10
1,5.224384e+10,33dfbd-b87a-4708-9857,2019-01-10 10:22:00,Electronic Appliances,Speakers,search,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-10,3,10
2,5.731416e+10,6511c2-e2e3-422b-b695,2019-01-10 14:00:00,Men's Fashion,Jeans,search,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-10,3,10
3,5.731416e+10,90fc70-0e80-4590-96f3,2019-01-10 14:07:00,Men's Fashion,Jeans,product_view,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-10,3,10
4,5.731416e+10,bd7419-2748-4c56-95b4,2019-01-10 14:12:00,Men's Fashion,Jeans,read_reviews,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-10,3,10


In [150]:
#Creating a base of people who have at least added an item to the cart (trigger point)
df_base = (df[df['Action'] == 'add_to_cart'].groupby('User_id').agg({'Category': 'max', 'SubCategory' : 'max'}).reset_index())
print(df_base.shape)
df_base.head()

(240, 3)


,User_id,Category,SubCategory
0,5.144816e+10,Camera Accessories,Camera Lens
1,5.145551e+10,Men's Fashion,Shoes
2,5.146260e+10,Mobile & Accessories,USB Cord
3,5.146362e+10,Stationary,Paints
4,5.146553e+10,Home Decor,Wall hangings


In [151]:
#adding days_active to the dataset.
#Why:

df_base = pd.merge(df_base, days_active, on = 'User_id', how = 'left')
print(df_base.shape)
df_base.head()

(240, 4)


,User_id,Category,SubCategory,no_of_days_active
0,5.144816e+10,Camera Accessories,Camera Lens,3
1,5.145551e+10,Men's Fashion,Shoes,3
2,5.146260e+10,Mobile & Accessories,USB Cord,5
3,5.146362e+10,Stationary,Paints,3
4,5.146553e+10,Home Decor,Wall hangings,5


In [152]:
#Adding RFM features
rfm_feats = df.groupby('User_id').agg({'R': 'max', 'F': 'max','M': 'max','Loyalty_Level': 'max', }).reset_index()
df_base = pd.merge(df_base, rfm_feats, on = 'User_id', how = 'left')
print(df_base.shape)
df_base.head()


(240, 8)


,User_id,Category,SubCategory,no_of_days_active,R,F,M,Loyalty_Level
0,5.144816e+10,Camera Accessories,Camera Lens,3,4.0,4.0,4.0,Platinum
1,5.145551e+10,Men's Fashion,Shoes,3,3.0,1.0,3.0,Gold
2,5.146260e+10,Mobile & Accessories,USB Cord,5,4.0,4.0,1.0,Platinum
3,5.146362e+10,Stationary,Paints,3,4.0,4.0,1.0,Platinum
4,5.146553e+10,Home Decor,Wall hangings,5,4.0,4.0,4.0,Platinum


In [153]:
#Adding average time between purchase

df_base = pd.merge(df_base, user_purch_date[['User_id', 'avg_time_between_purchase']], on='User_id', how = 'left')
print(df_base.shape)
df_base.head()

(240, 9)


,User_id,Category,SubCategory,no_of_days_active,R,F,M,Loyalty_Level,avg_time_between_purchase
0,5.144816e+10,Camera Accessories,Camera Lens,3,4.0,4.0,4.0,Platinum,184.0
1,5.145551e+10,Men's Fashion,Shoes,3,3.0,1.0,3.0,Gold,NaN
2,5.146260e+10,Mobile & Accessories,USB Cord,5,4.0,4.0,1.0,Platinum,56.0
3,5.146362e+10,Stationary,Paints,3,4.0,4.0,1.0,Platinum,29.0
4,5.146553e+10,Home Decor,Wall hangings,5,4.0,4.0,4.0,Platinum,7.0


In [154]:
df_base = pd.merge(df_base, cart_purch_ratio[['User_id', 'no_of_carts_to_purchase_ratio']], on= 'User_id', how = 'left')
print(df_base.shape)
df_base.head()

(240, 10)


,User_id,Category,SubCategory,no_of_days_active,R,F,M,Loyalty_Level,avg_time_between_purchase,no_of_carts_to_purchase_ratio
0,5.144816e+10,Camera Accessories,Camera Lens,3,4.0,4.0,4.0,Platinum,184.0,0.0
1,5.145551e+10,Men's Fashion,Shoes,3,3.0,1.0,3.0,Gold,NaN,0.0
2,5.146260e+10,Mobile & Accessories,USB Cord,5,4.0,4.0,1.0,Platinum,56.0,0.0
3,5.146362e+10,Stationary,Paints,3,4.0,4.0,1.0,Platinum,29.0,0.0
4,5.146553e+10,Home Decor,Wall hangings,5,4.0,4.0,4.0,Platinum,7.0,0.0


In [155]:
df_base = pd.merge(df_base, click_wishlist_purch_ratio_user[['User_id', 'no_of_click_wishlist_to_purchase_ratio']], on= 'User_id', how = 'left')
df_base = pd.merge(df_base, add_wishlist_purch_ratio_user[['User_id', 'no_of_add_wishlist_to_purchase_ratio']], on= 'User_id', how = 'left')
print(df_base.shape)
df_base.head()

(240, 12)


,User_id,Category,SubCategory,no_of_days_active,R,F,M,Loyalty_Level,avg_time_between_purchase,no_of_carts_to_purchase_ratio,no_of_click_wishlist_to_purchase_ratio,no_of_add_wishlist_to_purchase_ratio
0,5.144816e+10,Camera Accessories,Camera Lens,3,4.0,4.0,4.0,Platinum,184.0,0.0,0.0,NaN
1,5.145551e+10,Men's Fashion,Shoes,3,3.0,1.0,3.0,Gold,NaN,0.0,NaN,NaN
2,5.146260e+10,Mobile & Accessories,USB Cord,5,4.0,4.0,1.0,Platinum,56.0,0.0,NaN,NaN
3,5.146362e+10,Stationary,Paints,3,4.0,4.0,1.0,Platinum,29.0,0.0,0.0,0.0
4,5.146553e+10,Home Decor,Wall hangings,5,4.0,4.0,4.0,Platinum,7.0,0.0,0.0,NaN


In [156]:
df_base = pd.merge(df_base, temp_path[['User_id', 'top_paths']], on= 'User_id', how = 'left')
print(df_base.shape)
df_base.head()

(240, 13)


,User_id,Category,SubCategory,no_of_days_active,R,F,M,Loyalty_Level,avg_time_between_purchase,no_of_carts_to_purchase_ratio,no_of_click_wishlist_to_purchase_ratio,no_of_add_wishlist_to_purchase_ratio,top_paths
0,5.144816e+10,Camera Accessories,Camera Lens,3,4.0,4.0,4.0,Platinum,184.0,0.0,0.0,NaN,"[search, read_reviews, add_to_cart]"
1,5.145551e+10,Men's Fashion,Shoes,3,3.0,1.0,3.0,Gold,NaN,0.0,NaN,NaN,"[product_view, read_reviews, add_to_cart]"
2,5.146260e+10,Mobile & Accessories,USB Cord,5,4.0,4.0,1.0,Platinum,56.0,0.0,NaN,NaN,"[search, product_view, add_to_cart]"
3,5.146362e+10,Stationary,Paints,3,4.0,4.0,1.0,Platinum,29.0,0.0,0.0,0.0,"[search, product_view, add_to_cart]"
4,5.146553e+10,Home Decor,Wall hangings,5,4.0,4.0,4.0,Platinum,7.0,0.0,0.0,NaN,others


In [157]:
df_base = pd.merge(df_base, cart_purch_ratio_category[['Category', 'category_no_of_carts_to_purchase_ratio']], on= 'Category', how = 'left')
df_base = pd.merge(df_base, cart_purch_ratio_subcategory[['SubCategory', 'subcategory_no_of_carts_to_purchase_ratio']], on= 'SubCategory', how = 'left')
print(df_base.shape)
df_base.head()

(240, 15)


,User_id,Category,SubCategory,no_of_days_active,R,F,M,Loyalty_Level,avg_time_between_purchase,no_of_carts_to_purchase_ratio,no_of_click_wishlist_to_purchase_ratio,no_of_add_wishlist_to_purchase_ratio,top_paths,category_no_of_carts_to_purchase_ratio,subcategory_no_of_carts_to_purchase_ratio
0,5.144816e+10,Camera Accessories,Camera Lens,3,4.0,4.0,4.0,Platinum,184.0,0.0,0.0,NaN,"[search, read_reviews, add_to_cart]",0.0,0.0
1,5.145551e+10,Men's Fashion,Shoes,3,3.0,1.0,3.0,Gold,NaN,0.0,NaN,NaN,"[product_view, read_reviews, add_to_cart]",0.0,0.0
2,5.146260e+10,Mobile & Accessories,USB Cord,5,4.0,4.0,1.0,Platinum,56.0,0.0,NaN,NaN,"[search, product_view, add_to_cart]",0.0,0.0
3,5.146362e+10,Stationary,Paints,3,4.0,4.0,1.0,Platinum,29.0,0.0,0.0,0.0,"[search, product_view, add_to_cart]",0.0,0.0
4,5.146553e+10,Home Decor,Wall hangings,5,4.0,4.0,4.0,Platinum,7.0,0.0,0.0,NaN,others,0.0,0.0


In [158]:
df_base = pd.merge(df_base, click_wishlist_purch_ratio_category[['Category', 'category_no_of_click_wishilist_to_purchase_ratio']], on= 'Category', how = 'left')
df_base = pd.merge(df_base, click_wishlist_purch_ratio_subcategory[['SubCategory', 'subcategory_no_of_click_wishilist_to_purchase_ratio']], on= 'SubCategory', how = 'left')

df_base = pd.merge(df_base, add_wishlist_purch_ratio_category[['Category', 'category_no_of_add_wishilist_to_purchase_ratio']], on= 'Category', how = 'left')
df_base = pd.merge(df_base, add_wishlist_purch_ratio_subcategory[['SubCategory', 'subcategory_no_of_add_wishilist_to_purchase_ratio']], on= 'SubCategory', how = 'left')

print(df_base.shape)
df_base.head()

(240, 19)


,User_id,Category,SubCategory,no_of_days_active,R,F,M,Loyalty_Level,avg_time_between_purchase,no_of_carts_to_purchase_ratio,no_of_click_wishlist_to_purchase_ratio,no_of_add_wishlist_to_purchase_ratio,top_paths,category_no_of_carts_to_purchase_ratio,subcategory_no_of_carts_to_purchase_ratio,category_no_of_click_wishilist_to_purchase_ratio,subcategory_no_of_click_wishilist_to_purchase_ratio,category_no_of_add_wishilist_to_purchase_ratio,subcategory_no_of_add_wishilist_to_purchase_ratio
0,5.144816e+10,Camera Accessories,Camera Lens,3,4.0,4.0,4.0,Platinum,184.0,0.0,0.0,NaN,"[search, read_reviews, add_to_cart]",0.0,0.0,0.0,NaN,NaN,NaN
1,5.145551e+10,Men's Fashion,Shoes,3,3.0,1.0,3.0,Gold,NaN,0.0,NaN,NaN,"[product_view, read_reviews, add_to_cart]",0.0,0.0,0.0,0.0,0.0,NaN
2,5.146260e+10,Mobile & Accessories,USB Cord,5,4.0,4.0,1.0,Platinum,56.0,0.0,NaN,NaN,"[search, product_view, add_to_cart]",0.0,0.0,0.0,NaN,0.0,NaN
3,5.146362e+10,Stationary,Paints,3,4.0,4.0,1.0,Platinum,29.0,0.0,0.0,0.0,"[search, product_view, add_to_cart]",0.0,0.0,0.0,NaN,0.0,NaN
4,5.146553e+10,Home Decor,Wall hangings,5,4.0,4.0,4.0,Platinum,7.0,0.0,0.0,NaN,others,0.0,0.0,0.0,0.0,NaN,NaN


In [159]:
df_base = pd.merge(df_base, add_product_view_purch_ratio_category[['Category', 'category_no_of_product_view_to_purchase_ratio']], on= 'Category', how = 'left')
df_base = pd.merge(df_base, add_product_view_purch_ratio_subcategory[['SubCategory', 'subcategory_no_of_product_view_to_purchase_ratio']], on= 'SubCategory', how = 'left')

print(df_base.shape)
df_base.head()

(240, 21)


,User_id,Category,SubCategory,no_of_days_active,R,F,M,Loyalty_Level,avg_time_between_purchase,no_of_carts_to_purchase_ratio,...,no_of_add_wishlist_to_purchase_ratio,top_paths,category_no_of_carts_to_purchase_ratio,subcategory_no_of_carts_to_purchase_ratio,category_no_of_click_wishilist_to_purchase_ratio,subcategory_no_of_click_wishilist_to_purchase_ratio,category_no_of_add_wishilist_to_purchase_ratio,subcategory_no_of_add_wishilist_to_purchase_ratio,category_no_of_product_view_to_purchase_ratio,subcategory_no_of_product_view_to_purchase_ratio
0,5.144816e+10,Camera Accessories,Camera Lens,3,4.0,4.0,4.0,Platinum,184.0,0.0,...,NaN,"[search, read_reviews, add_to_cart]",0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0
1,5.145551e+10,Men's Fashion,Shoes,3,3.0,1.0,3.0,Gold,NaN,0.0,...,NaN,"[product_view, read_reviews, add_to_cart]",0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0
2,5.146260e+10,Mobile & Accessories,USB Cord,5,4.0,4.0,1.0,Platinum,56.0,0.0,...,NaN,"[search, product_view, add_to_cart]",0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0
3,5.146362e+10,Stationary,Paints,3,4.0,4.0,1.0,Platinum,29.0,0.0,...,0.0,"[search, product_view, add_to_cart]",0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0
4,5.146553e+10,Home Decor,Wall hangings,5,4.0,4.0,4.0,Platinum,7.0,0.0,...,NaN,others,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN


In [163]:
temp = (df.sort_values(by ='DateTime')[(df.DateTime == df.User_id.map(df[df['Action'] == 'purchase'].groupby('User_id').DateTime.max()))])
print(temp.shape)

purchase_users = temp[temp['Action'] == 'purchase'].groupby('User_id')['Action'].count().reset_index()
display(purchase_users.head())

df_base = pd.merge(df_base, purchase_users, on = 'User_id', how = 'left')
df_base['Action'] = df_base['Action'].fillna(0)
df_base.rename(columns = {'Action' : 'Target'}, inplace = True)
print(df_base.shape)
print(df_base.Target.value_counts())
df_base.head(3)


(179, 21)


,User_id,Action
0,5.144816e+10,1
1,5.145551e+10,1
2,5.146260e+10,1
3,5.146362e+10,1
4,5.146553e+10,1


(240, 22)
1.0    147
0.0     93
Name: Target, dtype: int64


,User_id,Category,SubCategory,no_of_days_active,R,F,M,Loyalty_Level,avg_time_between_purchase,no_of_carts_to_purchase_ratio,...,top_paths,category_no_of_carts_to_purchase_ratio,subcategory_no_of_carts_to_purchase_ratio,category_no_of_click_wishilist_to_purchase_ratio,subcategory_no_of_click_wishilist_to_purchase_ratio,category_no_of_add_wishilist_to_purchase_ratio,subcategory_no_of_add_wishilist_to_purchase_ratio,category_no_of_product_view_to_purchase_ratio,subcategory_no_of_product_view_to_purchase_ratio,Target
0,5.144816e+10,Camera Accessories,Camera Lens,3,4.0,4.0,4.0,Platinum,184.0,0.0,...,"[search, read_reviews, add_to_cart]",0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,1.0
1,5.145551e+10,Men's Fashion,Shoes,3,3.0,1.0,3.0,Gold,NaN,0.0,...,"[product_view, read_reviews, add_to_cart]",0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,1.0
2,5.146260e+10,Mobile & Accessories,USB Cord,5,4.0,4.0,1.0,Platinum,56.0,0.0,...,"[search, product_view, add_to_cart]",0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,1.0


In [164]:
df_base.isna().sum()

User_id                                                  0
Category                                                 0
SubCategory                                              0
no_of_days_active                                        0
R                                                       93
F                                                       93
M                                                       93
Loyalty_Level                                           93
avg_time_between_purchase                              208
no_of_carts_to_purchase_ratio                            0
no_of_click_wishlist_to_purchase_ratio                  99
no_of_add_wishlist_to_purchase_ratio                   217
top_paths                                                0
category_no_of_carts_to_purchase_ratio                   0
subcategory_no_of_carts_to_purchase_ratio                0
category_no_of_click_wishilist_to_purchase_ratio        25
subcategory_no_of_click_wishilist_to_purchase_ratio    1

### **Strategy:**

**R,F,M --> Zero imputation (RFM score of 0 indicated new user)**


**avg_time_between_purchase --> 1 imputation (since most users in the data is first time this feature has more NaN values)**


**Rest of features --> zero imputation**

In [259]:
df_base['avg_time_between_purchase'] = df_base['avg_time_between_purchase'].fillna(-1)
df_base = df_base.fillna(0)
df_base.isna().sum()

User_id                                                0
Category                                               0
SubCategory                                            0
no_of_days_active                                      0
R                                                      0
F                                                      0
M                                                      0
Loyalty_Level                                          0
avg_time_between_purchase                              0
no_of_carts_to_purchase_ratio                          0
no_of_click_wishlist_to_purchase_ratio                 0
no_of_add_wishlist_to_purchase_ratio                   0
top_paths                                              0
category_no_of_carts_to_purchase_ratio                 0
subcategory_no_of_carts_to_purchase_ratio              0
category_no_of_click_wishilist_to_purchase_ratio       0
subcategory_no_of_click_wishilist_to_purchase_ratio    0
category_no_of_add_wishilist_to

## Building the Propensity Model


A propensity model, also known as a predictive propensity model or propensity scoring model, is a statistical or machine learning model used to predict the likelihood or propensity of an individual or entity to take a specific action or exhibit a particular behavior. This action or behavior can vary widely, such as making a purchase, clicking on an ad, subscribing to a service, or responding to a marketing campaign. Propensity models leverage historical data and various features or variables to estimate the probability of the desired outcome, enabling businesses to target resources, marketing efforts, and interventions more effectively toward individuals or groups most likely to engage in the desired action.

In [290]:
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


In [291]:
df_model = df_base.copy()
print(df_model.shape)
df_model.head()

(240, 22)


,User_id,Category,SubCategory,no_of_days_active,R,F,M,Loyalty_Level,avg_time_between_purchase,no_of_carts_to_purchase_ratio,...,top_paths,category_no_of_carts_to_purchase_ratio,subcategory_no_of_carts_to_purchase_ratio,category_no_of_click_wishilist_to_purchase_ratio,subcategory_no_of_click_wishilist_to_purchase_ratio,category_no_of_add_wishilist_to_purchase_ratio,subcategory_no_of_add_wishilist_to_purchase_ratio,category_no_of_product_view_to_purchase_ratio,subcategory_no_of_product_view_to_purchase_ratio,Target
0,5.144816e+10,Camera Accessories,Camera Lens,3,4.0,4.0,4.0,Platinum,184.0,0.0,...,"[search, read_reviews, add_to_cart]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,5.145551e+10,Men's Fashion,Shoes,3,3.0,1.0,3.0,Gold,-1.0,0.0,...,"[product_view, read_reviews, add_to_cart]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,5.146260e+10,Mobile & Accessories,USB Cord,5,4.0,4.0,1.0,Platinum,56.0,0.0,...,"[search, product_view, add_to_cart]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,5.146362e+10,Stationary,Paints,3,4.0,4.0,1.0,Platinum,29.0,0.0,...,"[search, product_view, add_to_cart]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,5.146553e+10,Home Decor,Wall hangings,5,4.0,4.0,4.0,Platinum,7.0,0.0,...,others,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [292]:
df_model.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 240 entries, 0 to 239
Data columns (total 22 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   User_id                                              240 non-null    float64
 1   Category                                             240 non-null    object 
 2   SubCategory                                          240 non-null    object 
 3   no_of_days_active                                    240 non-null    int64  
 4   R                                                    240 non-null    float64
 5   F                                                    240 non-null    float64
 6   M                                                    240 non-null    float64
 7   Loyalty_Level                                        240 non-null    object 
 8   avg_time_between_purchase                            240 non-null    f

In [293]:
print(len(set([item for sublist in df_model['top_paths'] for item in sublist])))


13


In [295]:
df_model['top_paths'] = df_model['top_paths'].astype(str)


cat_cols = ['top_paths']
num_cols = df_model.drop(['Target', 'User_id'], axis=1).select_dtypes(['int', 'float']).columns.tolist()

cat_df = pd.get_dummies(df_model[cat_cols], prefix=cat_cols)

scaler = MinMaxScaler()
num_df = pd.DataFrame(scaler.fit_transform(df_model[num_cols]), columns=num_cols)

final_df = pd.concat([num_df, cat_df, df_model[['Target']]], axis=1)
print(final_df.shape)
final_df.head(3)




(240, 28)


,no_of_days_active,R,F,M,avg_time_between_purchase,no_of_carts_to_purchase_ratio,no_of_click_wishlist_to_purchase_ratio,no_of_add_wishlist_to_purchase_ratio,category_no_of_carts_to_purchase_ratio,subcategory_no_of_carts_to_purchase_ratio,...,"top_paths_['click_wishlist_page', 'add_to_cart']","top_paths_['product_view', 'click_wishlist_page', 'add_to_cart']","top_paths_['product_view', 'read_reviews', 'add_to_cart']","top_paths_['read_reviews', 'add_to_cart']","top_paths_['read_reviews', 'click_wishlist_page', 'add_to_cart']","top_paths_['search', 'click_wishlist_page', 'add_to_cart']","top_paths_['search', 'product_view', 'add_to_cart']","top_paths_['search', 'read_reviews', 'add_to_cart']",top_paths_others,Target
0,0.285714,1.00,1.00,1.00,0.748988,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,1.0
1,0.285714,0.75,0.25,0.75,0.000000,0.0,0.0,0.0,0.0,0.0,...,0,0,1,0,0,0,0,0,0,1.0
2,0.571429,1.00,1.00,0.25,0.230769,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,1.0


In [304]:
X = final_df.drop(['Target'],axis =1)
Y = final_df[['Target']]
x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size =0.3)
print((x_train.shape,y_train.shape), (x_test.shape, y_test.shape))

((168, 27), (168, 1)) ((72, 27), (72, 1))


In [303]:
print(y_train.value_counts())
print(y_train.value_counts(normalize = True))

Target
1.0       98
0.0       70
dtype: int64
Target
1.0       0.583333
0.0       0.416667
dtype: float64


In [298]:
logreg = LogisticRegression(class_weight = 'balanced', random_state =123)
logreg.fit(x_train, y_train)
print("Training Accuracy: " + str(logreg.score(x_test, y_test)))

Training Accuracy: 0.9861111111111112


In [305]:
preds = logreg.predict_proba(x_test)[:,1]
print(preds)
print("Test ROC-AUC: " + str(metrics.roc_auc_score(y_test, preds)))

[0.96968209 0.94633536 0.99203732 0.0996621  0.08359448 0.91543221
 0.61852763 0.0996621  0.9887492  0.06974956 0.08359448 0.89644759
 0.99311994 0.98630374 0.59851252 0.85840887 0.84841682 0.06974956
 0.96968209 0.06974956 0.11273643 0.98938023 0.99184844 0.70534078
 0.92425754 0.98630374 0.06974956 0.94102453 0.0996621  0.74777936
 0.88965815 0.08359448 0.98555674 0.06974956 0.61852763 0.95522814
 0.99165009 0.99311994 0.06974956 0.94312127 0.94758938 0.11273643
 0.08359448 0.06974956 0.99830715 0.85225614 0.83508542 0.0996621
 0.06974956 0.06974956 0.06974956 0.98946518 0.97047134 0.14522343
 0.60389813 0.99143559 0.96803022 0.83508542 0.11273643 0.95149364
 0.87139849 0.79622232 0.97928335 0.06974956 0.77157875 0.91971312
 0.99352225 0.89851432 0.96968209 0.94312127 0.09712641 0.99839562]
Test ROC-AUC: 1.0


In [306]:
preds_label = logreg.predict(x_test)
print(preds_label)
print("Test Accuracy: " + str(metrics.accuracy_score(y_test, preds_label)))

[1. 1. 1. 0. 0. 1. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0. 1. 1. 1.
 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 1. 0. 0. 0. 1. 1. 1. 0.
 0. 0. 0. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1.]
Test Accuracy: 1.0


In [309]:
pd.DataFrame(zip(x_train.columns, logreg.coef_[0]), columns = ['Feats', 'Imp']).sort_values(by = "Imp", ascending=False)

,Feats,Imp
1,R,3.518182
3,M,3.182052
2,F,2.076969
16,top_paths_['add_to_cart'],0.660708
0,no_of_days_active,0.521455
26,top_paths_others,0.255849
21,"top_paths_['read_reviews', 'add_to_cart']",0.211958
17,"top_paths_['checkout', 'purchase', 'add_to_cart']",0.174323
25,"top_paths_['search', 'read_reviews', 'add_to_c...",0.156470
4,avg_time_between_purchase,0.073376
